In [7]:
import os
import pandas as pd
import datetime
from supabase import create_client, Client
from GetData import preprocess_csv
from SupabaseHandle import request_table

# 엔드포인트 URL 및 API 키 설정
SUPABASE_URL = 'https://yhayrbotkkuuvoxzhqct.supabase.co'
SUPABASE_KEY = 'sb_secret_5rUltxbkuiB3wFcTyMs1qw_cJHFo3kf'

# 클라이언트 연결
try:
    supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("Supabase 클라이언트 연결 완료.")
except Exception as e:
    print(f"Supabase 클라이언트 연결 실패: {e}")

def validation(response):
    if response.data:
        print("데이터베이스 적용 완료")
    else:
        print("코드를 재확인해주세요.")

top_10_stocks = [
    {'name': '삼성전자', 'code': '005930.KS'},
    {'name': 'SK하이닉스', 'code': '000660.KS'},
    {'name': 'LG에너지솔루션', 'code': '373220.KS'},
    {'name': '삼성바이오로직스', 'code': '207940.KS'},
    {'name': '한화에어로스페이스', 'code': '012450.KS'},
    {'name': '삼성전자우', 'code': '005935.KS'},
    {'name': '현대차', 'code': '005380.KS'},
    {'name': 'KB금융', 'code': '105560.KS'},
    {'name': '두산에너빌리티', 'code': '034020.KS'},
    {'name': 'HD현대중공업', 'code': '329180.KS'},
    {'name': '휴켐스', 'code': '069260.KS'}
]


Supabase 클라이언트 연결 완료.


In [3]:
stock_codes = [stock['code'][:-3] for stock in top_10_stocks]

table = supabase.from_('technical_data').select('*').execute()

# empty data length = 0
len(table.data)
print(stock_codes)

['005930', '000660', '373220', '207940', '012450', '005935', '005380', '105560', '034020', '329180', '069260']


In [13]:
def request_table(table_name: str):
    """ Get All Rows in supabase """
    all_data = []
    page_size = 1000
    page = 0

    while True:
        start_index = page * page_size
        end_index = start_index + page_size - 1

        # print(f"{page + 1}번째 페이지 데이터 요청 중 (범위: {start_index} - {end_index})...")

        # range()를 사용하여 페이지별로 데이터 요청
        response = (supabase.from_(table_name).select('*').range(start_index, end_index).execute())

        data = response.data

        if not data:
            print("No Data")

        all_data.extend(data)

        # 가져온 데이터가 페이지 크기보다 작으면 마지막 페이지이므로 종료
        if len(data) < page_size:
            print("모든 데이터를 성공적으로 가져왔습니다.")
            break

        page += 1

    df = pd.DataFrame(all_data)

    # Data Preprocess
    df['stock_code'] = df['stock_code'].astype(str).str.zfill(6)

    return df

In [ ]:
def preprocess_csv(file_path):
    """ Load CSV >> 'stock_code' padding to zero """
    local_table = pd.read_csv(file_path)
    local_table['stock_code'] = local_table['stock_code'].astype(str).str.zfill(6)
    return local_table

In [4]:
def extract_unique_rows():
    """ Extract Unique rows >> Add new data"""
    # Filter Stock code rows
    get_table = request_table('technical_data')
    local_table = preprocess_csv('./data/stock_data.csv')

    # Uniform Column order
    table_columns = list(local_table.columns)
    get_table = get_table[table_columns]

    # Extract Unique rows
    new_data_rows = (
        pd.concat([get_table, local_table]).reset_index(drop = True)
        .drop_duplicates(subset = ['Date', 'stock_code'], keep = False)
        )

    return new_data_rows

In [25]:
f_df = add_new_data()

f_df.tail()

모든 데이터를 성공적으로 가져왔습니다.


,Date,stock_code,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_5,...,Bollinger_Upper,Bollinger_Lower,RSI,%K,%D,ADX,+DI,-DI,ATR,OBV
46483,2025-03-17,329180,307780.77,315749.2,301804.44,313757.09,356056,0.0,0.0,303398.14,...,334631.98,281292.38,59.00,78.95,65.35,12.26,22.96,16.99,15130.69,7460187059
46582,2025-08-08,329180,483000.00,483000.0,463500.00,466500.00,177577,0.0,0.0,467100.00,...,508007.18,373942.82,64.66,57.53,60.97,36.83,26.75,19.07,23443.76,7464738729


In [17]:
def extract_unique_rows():
    """ Extract Unique rows """
    # Filter Stock code rows
    get_table = request_table('technical_data')
    local_table = preprocess_csv('./stock_data.csv')

    # Extract Common Columns
    t_columns = set(get_table.columns)
    l_columns = set(local_table)
    columns = list(t_columns & l_columns)

    # Uniform Column order
    table_columns = list(local_table.columns)
    get_table = get_table[table_columns]

    # Extract Unique rows
    new_data_rows = (
        pd.concat([get_table[columns], local_table[columns]]).reset_index(drop = True)
        .drop_duplicates(subset = ['Date', 'stock_code'], keep = False)
        )

    return new_data_rows, get_table, local_table

In [18]:
a, b, c = extract_unique_rows()


모든 데이터를 성공적으로 가져왔습니다.


/var/folders/kp/n3m9dn7x50sd_fmhvr_v79x40000gn/T/ipykernel_78146/1703760824.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([get_table[columns], local_table[columns]]).reset_index(drop = True)


In [21]:
a

,Date,Bollinger_Lower,RSI,OBV,EMA_12,MACD_Signal,%K,Bollinger_Mid,Low,+DI,...,Bollinger_Upper,EMA_26,Open,Volume,SMA_5,ATR,stock_code,%D,ADX,MACD


In [19]:
b

,Date,stock_code,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_5,...,Bollinger_Upper,Bollinger_Lower,RSI,%K,%D,ADX,+DI,-DI,ATR,OBV
0,2000-01-04,005930,4245.211647,4323.040527,4004.649654,4323.040527,74195000,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
1,2000-01-05,005930,4103.705906,4287.665136,3905.595966,3948.048096,74680000,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
2,2000-01-06,005930,4068.328977,4089.555041,3948.047946,3976.349365,54390000,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
3,2000-01-07,005930,3933.897537,4011.726445,3792.390431,3919.746826,40305000,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
4,2000-01-10,005930,3962.198941,4082.479980,3948.048231,4082.479980,46880000,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46578,2025-08-04,329180,466000.000000,470000.000000,451000.000000,463500.000000,230145,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
46579,2025-08-05,329180,467500.000000,477500.000000,464000.000000,464000.000000,208970,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
46580,2025-08-06,329180,459500.000000,480500.000000,451500.000000,466500.000000,247270,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None
46581,2025-08-07,329180,474500.000000,486750.000000,471000.000000,475000.000000,259830,0.0,0.0,None,...,None,None,None,None,None,None,None,None,None,None


In [20]:
c

,Date,stock_code,Open,High,Low,Close,Volume,Dividends,Stock Splits,SMA_5,...,Bollinger_Upper,Bollinger_Lower,RSI,%K,%D,ADX,+DI,-DI,ATR,OBV
0,2000-01-04,005930,4245.21,4323.04,4004.65,4323.04,74195000,0.0,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,318.39,0
1,2000-01-05,005930,4103.71,4287.67,3905.60,3948.05,74680000,0.0,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,100.00,0.00,3.98,331.60,-74680000
2,2000-01-06,005930,4068.33,4089.56,3948.05,3976.35,54390000,0.0,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,100.00,0.00,3.74,306.25,-20290000
3,2000-01-07,005930,3933.90,4011.73,3792.39,3919.75,40305000,0.0,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,100.00,0.00,10.41,294.66,-60595000
4,2000-01-10,005930,3962.20,4082.48,3948.05,4082.48,46880000,0.0,0.0,4049.93,...,0.00,0.00,0.00,0.00,0.00,93.02,3.40,9.59,277.07,-13715000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46578,2025-08-04,329180,466000.00,470000.00,451000.00,463500.00,230145,0.0,0.0,475000.00,...,495746.61,356303.39,72.40,60.08,71.45,44.74,34.05,15.42,25496.54,7464080048
46579,2025-08-05,329180,467500.00,477500.00,464000.00,464000.00,208970,0.0,0.0,473100.00,...,499215.25,360284.75,71.19,58.47,63.31,44.49,35.57,14.22,23963.67,7464289018
46580,2025-08-06,329180,459500.00,480500.00,451500.00,466500.00,247270,0.0,0.0,472200.00,...,502619.39,364030.61,72.15,60.09,59.55,41.63,29.99,18.75,24635.18,7464536288
46581,2025-08-07,329180,474500.00,486750.00,471000.00,475000.00,259830,0.0,0.0,469100.00,...,506921.99,367178.01,72.79,65.30,61.29,39.92,30.09,16.65,24050.49,7464796118
